# Checklist
- [x] LMS -> create working repo.
- [ ] write inference code 
- [ ] setup uvicorn for localhost connection
- [ ] write request func. in dart
- [ ] connect func. with widget

In [2]:
#%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

---
# FastAPI setup
- Install faseAPI & uvicorn

In [1]:
#!pip install fastapi uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.3/395.3 KB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: anyio
    Found existing installation: anyio 3.5.0
    Uninstalling anyio-3.5.0:
      Successfully uninstalled anyio-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

- Uvicorn 서버로 통신 테스트

In [10]:
%%writefile api_test.py
import uvicorn   # pip install uvicorn 
from fastapi import FastAPI   # pip install fastapi 
import random

# Create the FastAPI application
app = FastAPI()

# A simple example of a GET request
@app.get("/")
def read_root():
    return "Hello World!"

@app.get('/example')
def root():
    return {"example":"This is example", "data": 23}

@app.get('/example/random')
def get_random():
    rn = random.randint(0,100)
    return {"number": rn, "limit": "100"}

@app.get('/example/random/{limit}')
def get_random(limit:int):
    rn = random.randint(0,limit)
    return {"number": rn, "limit": limit}

# Run the server
if __name__ == "__main__":
    uvicorn.run("api_test:app",
            reload= True,   # Reload the server when code changes
            host="127.0.0.1",   # Listen on localhost 
            port=5000,   # Listen on port 5000 
            log_level="info"   # Log level
            )

Overwriting api_test.py


---

# 모델 추론용 스크립트
- 입력 이미지 다운로드
- 가중치 다운로드
- VGG 모델 추론 코드
- API 형식으로 추론 실행

In [11]:
# 모델 다운받기

#사전학습 모델 불러오기 : 케라스에서 클래스 형태로 제공함 
from tensorflow.keras.applications.vgg16 import VGG16
 
#weight, include_top 파라미터 설정 
model = VGG16(weights='imagenet', include_top=True)
model.summary()

#save model
model.save('vgg16.h5')

553476096/553467096 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [12]:
%%writefile vgg16_prediction_model.py

# 예측에 필요한 라이브러리
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions

def prediction_model():
    img = Image.open('./sample_data/cat_224x224.jpg')
    print(type(img))
    model = tf.keras.models.load_model('vgg16.h5')
    # resize
    target_size = 224
    img = img.resize((target_size, target_size)) 

    #numpy array로 변경
    np_img = image.img_to_array(img)

    #4차원으로 변경 
    img_batch = np.expand_dims(np_img, axis=0)
    #feature normalization
    pre_processed = preprocess_input(img_batch)
    
    # 예측
    y_preds = model.predict(pre_processed)
    np.set_printoptions(suppress=True, precision=5) #소수 5자리까지 
    # 1위 예측 반환
    result = decode_predictions(y_preds, top=1)
    result = {"predicted_label" : str(result[0][0][1]), "prediction_score" :  str(result[0][0][2])}
    return result

Writing vgg16_prediction_model.py


In [23]:
%%writefile inference.py
import uvicorn   # pip install uvicorn 
from fastapi import FastAPI   # pip install fastapi 
import random

# 예측 모듈 가져오기
import vgg16_prediction_model

# Create the FastAPI application
app = FastAPI()

# A simple example of a GET request
@app.get("/")
def read_root():
    return "VGG16모델을 사용하는 API를 만들어봅시다."

@app.get('/sample')
async def sample_prediction():
    result = vgg16_prediction_model.prediction_model()  
    return result


# Run the server
if __name__ == "__main__":
    uvicorn.run("inference:app",
            reload= True,   # Reload the server when code changes
            host="127.0.0.1",   # Listen on localhost 
            port=5000,   # Listen on port 5000 
            log_level="info"   # Log level
            )

Writing inference.py


In [24]:
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions

def prediction_model():
    img = Image.open('./sample_data/cat_224x224.jpg')
    print(type(img))
    model = tf.keras.models.load_model('vgg16.h5')
    # resize
    target_size = 224
    img = img.resize((target_size, target_size)) 

    #numpy array로 변경
    np_img = image.img_to_array(img)

    #4차원으로 변경 
    img_batch = np.expand_dims(np_img, axis=0)
    #feature normalization
    pre_processed = preprocess_input(img_batch)
    
    # 예측
    y_preds = model.predict(pre_processed)
    np.set_printoptions(suppress=True, precision=5) #소수 5자리까지 
    # 1위 예측 반환
    result = decode_predictions(y_preds, top=1)
    result = {"predicted_label" : str(result[0][0][1]), "prediction_score" :  str(result[0][0][2])}
    return result

In [1]:
import vgg16_prediction_model
result2 = vgg16_prediction_model.prediction_model()

<class 'PIL.JpegImagePlugin.JpegImageFile'>


In [2]:
result2

{'predicted_label': 'Egyptian_cat', 'prediction_score': '0.2864011'}